# Code 48
ReAct agent with 3 tools, recursion limit fix, and stricter stop instructions.

In [ ]:

import json, os, csv
from typing import Dict, Any

from langchain.agents import create_react_agent
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder


In [ ]:

def summarize_patient_record(record_str: str) -> str:
    """Summarize a patient record JSON string into structured fields."""
    record = json.loads(record_str)
    summary = {
        "patient_id": record.get("patient_id"),
        "age": 2025 - int(record.get("date_of_birth", "1900-01-01").split("-")[0]),
        "sex": record.get("gender"),
        "diagnoses": record.get("diagnosis_codes", []),
        "procedures": record.get("procedure_codes", []),
        "policy_id": record.get("insurance_policy_id"),
        "preauth_required": record.get("preauthorization_required", False),
        "preauth_obtained": record.get("preauthorization_obtained", False),
    }
    return json.dumps(summary)

def summarize_policy_guideline(policy_id: str) -> str:
    """Summarize insurance policy rules for a given policy_id."""
    with open("./Data/insurance_policies.json") as f:
        policies = json.load(f)
    policy = next((p for p in policies if p["policy_id"] == policy_id), None)
    return json.dumps(policy if policy else {"error": f"Unknown policy_id {policy_id}"} )

def check_claim_coverage(record_summary: str, policy_summary: str) -> str:
    """Check claim coverage given a patient record summary and a policy summary."""
    record = json.loads(record_summary)
    policy = json.loads(policy_summary)

    if "error" in policy:
        return json.dumps({"decision": "ROUTE FOR REVIEW", "reason": policy["error"]})

    age = record.get("age")
    sex = record.get("sex")
    reasons = []

    for proc in record.get("procedures", []):
        match = None
        for cp in policy.get("covered_procedures", []):
            if cp["procedure_code"] == proc:
                match = cp
                break
        if not match:
            reasons.append(f"CPT {proc} is not covered by the policy.")
            continue

        if age is not None and "age_range" in match:
            min_age, max_age = match["age_range"]
            if age < min_age or age > max_age:
                reasons.append(f"Patient age {age} outside covered range for CPT {proc}.")

        if sex and match.get("gender") not in ("Any", None):
            if sex[0].upper() != match["gender"][0].upper():
                reasons.append(f"Policy restricted to sex {match['gender']}, patient sex is {sex}.")

        if match.get("requires_preauthorization", False) and not record.get("preauth_obtained"):
            reasons.append(f"Preauthorization required for CPT {proc} but not provided.")

        if not record.get("diagnoses"):
            reasons.append(f"No diagnosis provided for CPT {proc}.")
        else:
            if not any(dx in match.get("covered_diagnoses", []) for dx in record["diagnoses"]):
                reasons.append(f"CPT {proc} not covered for provided diagnosis codes.")

    if reasons:
        return json.dumps({"decision": "ROUTE FOR REVIEW", "reason": " ".join(reasons)})
    return json.dumps({"decision": "APPROVE", "reason": "Meets policy criteria."})


In [ ]:

# Build ReAct agent with stricter stop instructions
TOOLS = [summarize_patient_record, summarize_policy_guideline, check_claim_coverage]

SYSTEM_PROMPT = """You are an insurance claim evaluation agent.
You MUST always follow this sequence strictly:
1. Call summarize_patient_record
2. Call summarize_policy_guideline
3. Call check_claim_coverage
Then STOP and return the final decision and reason. Do not loop tools.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

agent = create_react_agent(chat_client, TOOLS, prompt)


In [ ]:

def run_agent_on_records(agent, records, n_debug=0) -> list[Dict[str, Any]]:
    results = []
    for i, rec in enumerate(records):
        if n_debug and i >= n_debug:
            break
        rec_str = json.dumps(rec)
        try:
            response = agent.invoke(
                {"input": f"Evaluate claim for patient record: {rec_str}"},
                config={"recursion_limit": 6}
            )
            parsed = json.loads(response)
            results.append({
                "patient_id": rec["patient_id"],
                "generated_response": f"- Decision: {parsed['decision']}\n- Reason: {parsed['reason']}"
            })
        except Exception as e:
            results.append({
                "patient_id": rec.get("patient_id", f"rec_{i}"),
                "generated_response": f"- Decision: ROUTE FOR REVIEW\n- Reason: Error {e}"
            })
    return results
